In [1]:
%matplotlib inline

from nddata import NDDataArray, DataAxis
import numpy as np

#Init histogram
hist = NDDataArray()

#Add x-axis
xaxis = DataAxis(np.linspace(0,100,10),'m', name='distance')
hist.add_axis(xaxis)
print('Dimensions: ',hist.dim)

('Dimensions: ', 1)


In [2]:
#Add faulty data
data = np.arange(15)
hist.data = data

ValueError: Data shape does not match in dimension 0
Axis "distance": 10, Data 15

In [3]:
# Add real data
import random
data = [random.expovariate(hist.axes[0][_].value+1) for _ in range(10)]
print(data)
hist.data=data

[0.609623070106417, 0.004155307113044101, 0.1774059556590868, 0.0037044323358520854, 0.008296547863930115, 0.02840890613694828, 0.023630942066698305, 0.00358458031695415, 0.017293338497348584, 0.009596083932415846]


In [4]:
# Find nodes
from astropy.units import Unit as u
hist.find_node(distance = [2627 * u('cm'), 57 * u('m')])

# Evaluate (no interpolation)
hist.evaluate_nearest(distance = [14 * u('m'), 87 * u('m')])

<Quantity [ 0.00415531, 0.01729334]>

In [5]:
# Evaluate using scipy interpolator
hist.add_linear_interpolator()
val1 = hist.evaluate(distance = [14 * u('m'), 87 * u('m')], method='nearest')
val2 = hist.evaluate(distance = [14 * u('m'), 87 * u('m')], method='linear')
print(val1, val2)

(<Quantity [ 0.00415531, 0.01729334]>, <Quantity [ 0.04920048, 0.01496285]>)


In [6]:
# Add second dimension
from nddata import BinnedDataAxis

yaxis = BinnedDataAxis.linspace(0, 5, 5, 'deg')
yaxis.name = 'angle'

hist.add_axis(yaxis)
print('Dimension', hist.dim)

# Data is deleted to avoid funcy behaviour
print(hist.data)

('Dimension', 2)
None


In [7]:
# Add faulty data (wrong axis order)
val = np.arange(1,6)
d = np.array(data)
data_2d = np.tensordot(d, val, axes=0)

print(data_2d.shape)
hist.data = data_2d

(10, 5)


ValueError: Data shape does not match in dimension 0
Axis "angle": 5, Data 10

In [12]:
# Add correct data
hist.data = np.transpose(data_2d)


In [15]:
# Interpolation

hist.add_linear_interpolator(bounds_error=True)

val1 = hist.evaluate(distance = 65 * u('m'), angle = 3 * u('deg'))

val2 = hist.evaluate(distance = [65, 23, 26] * u('m'), angle = 3 * u('deg'))

val3 = hist.evaluate(distance = 65 * u('m'))

print(val1.shape, val2.shape, val3.shape)

((2, 2), (1, 3), (5, 1))


In [16]:
# FITS I/O

test_file = 'test_file.fits'
hist.write(test_file, overwrite=True)
hist2 = NDDataArray.read(test_file)